<a href="https://colab.research.google.com/github/Jayplect/home-sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup Tools

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 108 kB in 2s (43.6 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.createOrReplaceTempView('home_sales')


# Determination of Key Metrics about the Home sale Data

In [5]:
# The average price for a four bedroom house sold in each year rounded to two decimal places.
query = ("""
SELECT ROUND(AVG(price),2) AS Avg_price,
    year(date) AS year 
    FROM home_sales
    WHERE bedrooms=4
    GROUP BY year
""")

spark.sql(query).show()

+---------+----+
|Avg_price|year|
+---------+----+
|296363.88|2022|
| 300263.7|2019|
|298353.78|2020|
|301819.44|2021|
+---------+----+



In [6]:
# The average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms
query = ("""
SELECT date_built AS Date_built, 
      ROUND(AVG(price), 2) AS Avg_Price
      FROM home_sales 
      WHERE bedrooms= 3 AND bathrooms=3
      GROUP BY date_built
      ORDER BY date_built
  """)

spark.sql(query).show()

+----------+---------+
|Date_built|Avg_Price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# The average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places
query = ("""
SELECT date_built AS Date_built, 
        ROUND(AVG(price),2) AS Avg_Price 
        FROM home_sales 
          WHERE bedrooms = 3 
              AND bathrooms = 3 
              AND floors = 2 
              AND sqft_living >= 2000
        GROUP BY date_built
        ORDER BY date_built
""")

spark.sql(query).show()

+----------+---------+
|Date_built|Avg_Price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# The "view" rating for the average price of a home, rounded to two decimal places, 
# where the homes are greater than or equal to $350,000.
# Determine the run time for this query.

start_time = time.time()

query = ("""
SELECT ROUND(AVG(view),2) AS View_Rating_Avg_Price
    FROM home_sales
    WHERE price >= 350000
  """)
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------------------+
|View_Rating_Avg_Price|
+---------------------+
|                32.26|
+---------------------+

--- 0.7496566772460938 seconds ---


In [9]:
# Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [10]:
# Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# Using the cached data, run the query that filters out the view ratings with average price 
# greater than or equal to $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = ("""
SELECT ROUND(AVG(view),2) AS View_Rating_Avg_Price
    FROM home_sales
    WHERE price >= 350000
  """)

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------------------+
|View_Rating_Avg_Price|
+---------------------+
|                32.26|
+---------------------+

--- 0.41808319091796875 seconds ---


In [12]:
# Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy("date_built").parquet('p_home_sales', mode='overwrite')

In [13]:
# Read the parquet formatted data.
parquet_df = spark.read.parquet('p_home_sales')

In [14]:
# Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('partitioned_home_sales')

In [15]:
# Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = ("""
SELECT ROUND(AVG(view),2) AS View_Rating_Avg_Price
    FROM home_sales
    WHERE price >= 350000
  """)

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------------------+
|View_Rating_Avg_Price|
+---------------------+
|                32.26|
+---------------------+

--- 0.3482334613800049 seconds ---


In [16]:
# Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [17]:
# Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False